In [1]:
# import numpy as np
# import pandas as pd
# from tqdm.auto import tqdm
import os
# import matplotlib.pyplot as plt
# # pd.set_option('display.max_columns', None)
# from matplotlib import rc
# rc('text',usetex=True)
# rc('text.latex', preamble='\\usepackage{color}')

In [12]:
target_name = 'random_5_15_s5'
sym_graver_bin = '../build/sym_hilbert'

In [ ]:
os.makedirs(f'mats_{target_name}', exist_ok=True)
problem_list = []
for n in range(100):
    out_fname = f'mats_{target_name}/problem{n:03}.mat'
    os.remove(out_fname) if os.path.exists(out_fname) else None
    if not os.path.exists(out_fname):
        cmd = f'{sym_graver_bin} -unit-test-generate-matrix 5 15 {out_fname}'
        print(cmd)
        os.system(cmd)

    problem_list.append(out_fname.split('.')[0])

with open(f'models_{target_name}.txt', 'w') as f:
    for m in problem_list:
        f.write(m + '\n')

# Filter MCC models

In [7]:
import os
import subprocess
import hashlib

# Configurable limits
max_rows = 500  # Replace with your limit
max_cols = 500  # Replace with your limit
max_val = 10  # Replace with your limit

def hash_matrix(matrix):
    """Generates a hash for a matrix (list of lists)"""
    matrix_str = ''.join(map(str, [item for row in matrix for item in row]))
    return hashlib.sha256(matrix_str.encode('utf-8')).hexdigest()

def read_matrix_from_file(file_path):
    """Reads matrix data from the .mat file"""
    try:
        with open(file_path, 'r') as f:
            # First line is the number of rows
            rows, cols = list(map(int, f.readline().strip().split()))
            # Second line is the number of columns
            # cols = int(f.readline().strip())
            # Remaining lines are the matrix values
            matrix = []
            for _ in range(rows):
                row = list(map(int, f.readline().strip().split()))
                matrix.append(row)
        return rows, cols, matrix
    except:
        return None, None, None

def is_valid_matrix(matrix, max_rows, max_cols, max_val):
    """Check if the matrix meets the criteria for size and value limits"""
    if matrix is None:
        return False
    rows, cols = len(matrix), len(matrix[0])
    if rows > max_rows or cols > max_cols:
        return False
    for row in matrix:
        for val in row:
            if abs(val) > max_val:
                return False
    return True

def process_directories(root_folder):
    """Main function to process all */model.net files"""
    unique_hashes = {}
    unique_matrices = []
    
    # Walk through all subdirectories
    for subdir, dirs, files in os.walk(root_folder):
        for file in files:
            if file == 'model.net':
                model_net_path = os.path.join(subdir, file)

                with open(model_net_path, 'r') as f:
                    _ = f.readline()
                    _ = f.readline()
                    header = f.readline().strip().split()
                    nP, nT = header[2], header[4]

                print(model_net_path, nP, nT)

                if int(nP) > max_rows or int(nT) > max_cols:
                    continue
                                
                # Assuming the .mat file is saved in the same folder
                mat_file_path = os.path.join(subdir, 'model.mat')
                
                # Run the external command to generate the .mat file
                subprocess.run(['timeout', '1s', 'DSPN-Tool', '-load', os.path.join(subdir, 'model'), '-imTxP'])

                # Read the matrix from the .mat file
                if os.path.exists(mat_file_path):
                    rows, cols, matrix = read_matrix_from_file(mat_file_path)
                    
                    # Filter matrices based on criteria
                    if is_valid_matrix(matrix, max_rows, max_cols, max_val):
                        # Hash the matrix to identify uniqueness
                        matrix_hash = hash_matrix(matrix)
                        
                        if matrix_hash not in unique_hashes:
                            # Store the matrix only if it's unique
                            unique_hashes[matrix_hash] = matrix
                            unique_matrices.append(mat_file_path)
                            print(f"Unique matrix found in: {mat_file_path}")

                        else:
                            os.remove(mat_file_path)
    
    return unique_matrices

# Run the function
root_folder = '/Users/elvio/MCC/2020/MODELS/'  # Replace with the root directory containing the problem folders
unique_matrices = process_directories(root_folder)

# Output result
print(f"Total unique matrices found: {len(unique_matrices)}")

/Users/elvio/MCC/2020/MODELS/RwMutex-PT-r1000w0010/model.net 3020 2020
/Users/elvio/MCC/2020/MODELS/Sudoku-PT-AN06/model.net 324 216
LOADING PETRI NET /Users/elvio/MCC/2020/MODELS/Sudoku-PT-AN06/model (net/def)...
MARKING PAR: 0
PLACES:      324
RATE PAR:    0
TRANSITIONS: 216
MEASURES:    0
LOADING TIME: [User 0.001s, Sys 0.000s]


COMPUTING INCIDENCE...
M=324, N=216, N0=216
SAVING INCIDENCE MATRIX AS: /Users/elvio/MCC/2020/MODELS/Sudoku-PT-AN06/model.mat
Unique matrix found in: /Users/elvio/MCC/2020/MODELS/Sudoku-PT-AN06/model.mat
/Users/elvio/MCC/2020/MODELS/Referendum-PT-0010/model.net 31 21
LOADING PETRI NET /Users/elvio/MCC/2020/MODELS/Referendum-PT-0010/model (net/def)...
MARKING PAR: 0
PLACES:      31
RATE PAR:    0
TRANSITIONS: 21
MEASURES:    0
LOADING TIME: [User 0.000s, Sys 0.000s]


COMPUTING INCIDENCE...
M=31, N=21, N0=21
SAVING INCIDENCE MATRIX AS: /Users/elvio/MCC/2020/MODELS/Referendum-PT-0010/model.mat
Unique matrix found in: /Users/elvio/MCC/2020/MODELS/Referendum-PT

In [10]:
print(len(unique_matrices))
display(unique_matrices)

392


['/Users/elvio/MCC/2020/MODELS/Sudoku-PT-AN06/model.mat',
 '/Users/elvio/MCC/2020/MODELS/Referendum-PT-0010/model.mat',
 '/Users/elvio/MCC/2020/MODELS/ProductionCell-PT-none/model.mat',
 '/Users/elvio/MCC/2020/MODELS/DiscoveryGPU-PT-09b/model.mat',
 '/Users/elvio/MCC/2020/MODELS/ShieldRVt-PT-002A/model.mat',
 '/Users/elvio/MCC/2020/MODELS/DNAwalker-PT-07track28RR/model.mat',
 '/Users/elvio/MCC/2020/MODELS/Solitaire-PT-EngCT7x7/model.mat',
 '/Users/elvio/MCC/2020/MODELS/Sudoku-PT-AN01/model.mat',
 '/Users/elvio/MCC/2020/MODELS/ShieldPPPs-PT-005B/model.mat',
 '/Users/elvio/MCC/2020/MODELS/TriangularGrid-PT-3011/model.mat',
 '/Users/elvio/MCC/2020/MODELS/CloudOpsManagement-PT-00080by00040/model.mat',
 '/Users/elvio/MCC/2020/MODELS/GPPP-PT-C0001N0000000001/model.mat',
 '/Users/elvio/MCC/2020/MODELS/DiscoveryGPU-PT-06a/model.mat',
 '/Users/elvio/MCC/2020/MODELS/CircadianClock-PT-000010/model.mat',
 '/Users/elvio/MCC/2020/MODELS/FMS-PT-02000/model.mat',
 '/Users/elvio/MCC/2020/MODELS/Sudoku-